# Agents with Multi-Turn Conversation 

<h3> By Default Agents are stateless and do not maintain any internal state. But it might happen you need to maintain the state between two or three agents or Agents should know about its details</h3>
<h4>Here we will look into preparing a Masala Tea for our Agent how things works, with thread or without thread.</h4>

In [1]:
# Installation refer 'basic_food_agent.ipynb' , installation section
# !pip install -U agent-framework --pre

In [2]:
# Import core dependencies to create the agent, for Agent Framework
import asyncio
import os
import json

from dotenv import load_dotenv, find_dotenv
# Core components for building Agent, tool-enabled agents
from agent_framework import ChatAgent
from agent_framework.openai import OpenAIChatClient

In [3]:
# load environment file
load_dotenv(find_dotenv())

True

In [4]:
# Setup OpenAIChatClient for LLM Inference - Here we will use OpenRouter API which is compatible with OpenAI and NVIDIA 30B model
# This client connects to the OpenRouter Models which are OpenAI-compatible endpoint
# Environment variables required
# OPENROUTER_ENDPOINT - 
# OPENROUTER_API_KEY
openai_chat_client = OpenAIChatClient(
    base_url=os.environ.get("OPENROUTER_ENDPOINT"),
    api_key=os.environ.get("OPENROUTER_API_KEY"),
    model_id="nvidia/nemotron-3-nano-30b-a3b:free"
)

In [5]:
AGENT_NAME = "FoodAgent"
AGENT_INSTRUCTIONS = """ You are an helpful AI Agent that help customers prepare variety of foods and cuisines as per their taste. 
                         You are a very highly classified chef which also listens to customers recommendations.
                         Keep your response short, concise and meaningful with 200 words or less than that.
"""
# restricting the words to keep the response short for demo purpose

In [6]:
# create the agent remember we are not using any tools here, this is simple example
agent = ChatAgent(
    name = AGENT_NAME,
    chat_client=openai_chat_client,
    instructions=AGENT_INSTRUCTIONS
)

<h3> First we will see without thread or storing conversations, how it behaves </h3>

In [7]:
response1 = await agent.run("Prepare me masala tea with ginger and cardamom, it's my first order")

In [8]:
print("Food Order: ")
print(response1.text)

Food Order: 
Sure! Here’s a quick, flavor‑packed masala chai with ginger and cardamom—perfect for your first order:

**Ingredients (serves 1)**
- 1 cup water  
- 1 cup milk (whole or oat for creaminess)  
- 1 tsp loose black tea leaves (or 1 tea bag)  
- ½ tsp grated fresh ginger  
- 2‑3 green cardamom pods, lightly crushed  
- 1‑2 thin cinnamon sticks  
- 1 pinch black pepper (optional)  
- 1‑2 tsp honey or sugar (adjust to taste)  

**Method**
1. In a small saucepan, combine water, ginger, cardamom, and cinnamon. Bring to a gentle boil and simmer 2 min to infuse.  
2. Add the tea leaves (or bag) and black pepper; steep 1 min.  
3. Stir in milk and sugar/honey; heat until just below boiling—do **not** let it boil over.  
4. Strain into a mug, sweeten if needed, and enjoy hot.

**Tip**: For extra froth, whisk vigorously or use a milk frother before serving. Savor the warm spice blend—your perfect first masala chai is ready! 🌿🍵


<h3> Let's see if Agent remembers our first order or not </h3>

In [9]:

response2 = await agent.run("What was my first order?")

In [10]:
print(response2.text)

I’m not sure which specific order you’re referring to. Could you tell me the date, dish name, or any details about the order you’d like to know? Then I can tell you what you ordered.


<h3>Oops!!! Agent has no idea what was our first order </h3>

<h3> Now let's use the thread and check if our agent remembers order or not</h3>

In [11]:
thread = agent.get_new_thread() # creating the thread which holds on the conversation

In [12]:
response3 = await agent.run("Help me to prepare Green Tea, this is my second order", thread=thread)

In [13]:
print(response3.text)

Sure! Here’s a quick, fool‑proof Green Tea recipe:

**Ingredients**
- 1 tsp loose‑leaf green tea (or 1 tea bag)
- 180 ml (¾ cup) fresh water
- Optional: honey, lemon, mint, or a splash of milk

**Steps**
1. **Heat water** to 70‑80 °C (just below boiling). If you only have boiling water, let it sit 1‑2 min to cool.
2. **Pre‑warm** your cup or teapot – pour a little hot water, swirl, then discard.
3. **Add tea** to the vessel.
4. **Pour water** over the tea, steep for 2‑3 minutes (longer makes it bitter).
5. **Remove** tea leaves or bag. Adjust flavor: a drizzle of honey, a squeeze of lemon, or a sprig of mint if you like.
6. **Enjoy** immediately while it’s warm.

**Tips for the perfect cup**
- Never over‑steep; green tea is delicate.
- If you prefer a smoother taste, try a slightly lower temperature (65 °C) next time.
- Experiment with flavors—yuzu, ginger, or a dash of ginger syrup add a fresh twist.

Let me know how you’d like to tweak it, or if you need a larger batch!


In [14]:
response4 = await agent.run("What was my second order ?", thread=thread)


In [15]:
print(response4.text)

Your second order was **green tea**. Let me know if you’d like a different variation or additional details for your next preparation!


<h3> See, agent now remembers, our previous order was green tea </h3>